In [1605]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import json
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from itertools import zip_longest

In [1606]:
chrom_options = webdriver.ChromeOptions()
chrom_options.add_experimental_option('detach', True)
chrom_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
                          options=chrom_options)

In [ ]:
url = "https://map.naver.com/p/search/%EC%98%A4%EC%95%BC%EC%B8%A0%ED%83%80%EC%BD%94/place/1256573788?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp"
driver.get(url)
driver.implicitly_wait(5)

In [1608]:
WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe"))
)


True

In [1609]:
# 결과 저장용 DataFrame 생성
naver_restaurants = pd.DataFrame(columns=['식당명', '카테고리', '주소', '영업시간'])

In [1610]:
name_elements = driver.find_elements(By.CSS_SELECTOR,".GHAhO")
name_elements

[<selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.CE0C71C519BBD1D7F11D2F78727F8C81.d.1A24775C8716CDE75796E0D3A23E4755.e.119")>]

In [1611]:
STORE_NAME=[]
for i in name_elements:
    print(i.text)
    STORE_NAME.append(i.text)
naver_restaurants['식당명'] = STORE_NAME

바다이야기


In [1612]:
Category_element = driver.find_elements(By.CSS_SELECTOR,".lnJFt")
Category_element

[<selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.CE0C71C519BBD1D7F11D2F78727F8C81.d.1A24775C8716CDE75796E0D3A23E4755.e.120")>]

In [1613]:
STORE_Category=[]

for i in Category_element:
    print(i.text)
    name4csv=i.text
    STORE_Category.append(i.text)
naver_restaurants['카테고리'] = STORE_Category

조개요리


In [1614]:
driver.get(url)
driver.implicitly_wait(5)
WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe"))
)

True

In [1615]:
address_elements = driver.find_elements(By.CSS_SELECTOR,".LDgIH")
address_elements

[<selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.8481593E0EFF92B57CB0EAF8201DBC1E.d.A85800AC5865BA014471494160105409.e.231")>]

In [1616]:
STORE_Address=[]
for i in address_elements:
    print(i.text)
    STORE_Address.append(i.text)
naver_restaurants['주소'] = STORE_Address

대구 동구 동부로30길 42-1


In [1617]:
driver.get(url)
driver.implicitly_wait(5)
WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe"))
)

True

In [1618]:
buttons = driver.find_elements(By.CSS_SELECTOR, "._UCia")
buttons[1].click()

In [1619]:
Time_elements = driver.find_elements(By.CSS_SELECTOR,".y6tNq")
Time_elements

[<selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.4A28A326BCC328EAA26C2A7488D4C5B3.d.D5BFBCF323938611CB2D93B54E183E90.e.357")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.4A28A326BCC328EAA26C2A7488D4C5B3.d.D5BFBCF323938611CB2D93B54E183E90.e.358")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.4A28A326BCC328EAA26C2A7488D4C5B3.d.D5BFBCF323938611CB2D93B54E183E90.e.359")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.4A28A326BCC328EAA26C2A7488D4C5B3.d.D5BFBCF323938611CB2D93B54E183E90.e.360")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.4A28A326BCC328EAA26C2A7488D4C5B3.d.D5BFBCF323938611CB2D93B54E183E90.e.361")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a5906

In [1620]:
STORE_Time = []
temp_times = []  # 임시로 각 영업시간을 저장할 리스트

# Time_elements가 비어있거나 None인 경우 처리
if not Time_elements:
    STORE_Time.append(None)
else:
    for i in Time_elements:
        if i is None or not i.text:  # 요소가 None이거나 텍스트가 비어있는 경우
            continue
            
        time_text = i.text
        
        # 불필요한 텍스트 제거 (\n 포함)
        time_text = time_text.replace('영업 종료', '')\
                            .replace('영업시작', '')\
                            .replace('영업 시작', '')\
                            .replace('접기', '')\
                            .replace('\n', '')\
                            .strip()
        
        # 빈 문자열이 된 경우 건너뛰기
        if not time_text:
            continue
            
        # 시간 형식 통일 (09:00 형식으로)
        time_text = time_text.replace('9시 0분', '09:00')
         # "매일" 형식의 영업시간을 요일별로 변환
        if '매일' in time_text:
            # 예: "매일 11:30 - 24:00" -> "월 11:30 -24:00`화 11:30 -24:00`..."
            time_part = time_text.replace('매일', '').strip()
            days = ['월', '화', '수', '목', '금', '토', '일']
            for day in days:
                temp_times.append(f'{day} {time_part}')
        # 요일별 영업시간 형식 정리
        if '정기휴무' in time_text:
            # 정기휴무 정보는 그대로 유지
            temp_times.append(time_text.strip())
        else:
            # 영업시간 정보 정리 (예: 금09:00-22:00 형식)
            parts = time_text.split()
            if len(parts) >= 3:  # 요일과 시간 정보가 있는 경우
                day = parts[0]  # 요일
                times = ''.join(parts[1:]).replace(' ', '')  # 시간 정보에서 공백 제거
                temp_times.append(f'{day}{times}')  # 요일과 시간 사이 공백 없이

    # 수집된 시간 정보가 있는 경우에만 처리
    if temp_times:
        combined_time = '`'.join(temp_times)
        STORE_Time.append(combined_time)
    else:
        STORE_Time.append(None)

# DataFrame에 추가
naver_restaurants['영업시간'] = STORE_Time

# 결과 확인
print("정리된 영업시간:", STORE_Time[0])

정리된 영업시간: 영업중24:00에0시0분에`금17:00-24:00`토17:00-24:00`일정기휴무 (매주 일요일)`월17:00-24:00`화17:00-24:00`수17:00-24:00`목17:00-24:00`-월요일이공휴일이면일요일정상영업,월요일휴무.


In [1621]:
driver.get(url)
driver.implicitly_wait(5)
WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe"))
)

True

In [1622]:
# 버튼 요소들 찾기
buttons = driver.find_elements(By.CSS_SELECTOR, ".tpj9w._tab-menu")
# 각 버튼의 자식 텍스트 확인 후 클릭
for button in buttons:
    if "메뉴" in button.text:  # 버튼의 텍스트에 "메뉴" 포함 여부 확인
        button.click()  # 클릭 실행
        break  # 첫 번째 "메뉴" 버튼만 클릭하고 종료

In [1623]:
# 'fvwqf' 태그가 있는 버튼이 없어질 때까지 반복
while True:
    try:
        # 버튼 요소 찾기
        button = driver.find_element(By.CSS_SELECTOR, ".fvwqf")
        button.click()  # 버튼 클릭
        print("버튼 클릭 완료")
        
        # 스크롤 내리기
        driver.execute_script("window.scrollBy(0, 300);")  # 300px 아래로 스크롤
        print("스크롤 완료")
        
        time.sleep(1)  # 클릭 및 스크롤 후 잠시 대기 (필요에 따라 조정)
    except:
        # 버튼이 없으면 루프 종료
        print("버튼이 더 이상 존재하지 않습니다.")
        break


버튼이 더 이상 존재하지 않습니다.


In [1624]:
menu_elements = driver.find_elements(By.CSS_SELECTOR, ".yQlqY")
menu_elements

[<selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.671016C478D89C027E75B16AC2C6E280.d.ECDB5E67B2953CB3C7D31150D5479B75.e.480")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.671016C478D89C027E75B16AC2C6E280.d.ECDB5E67B2953CB3C7D31150D5479B75.e.481")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.671016C478D89C027E75B16AC2C6E280.d.ECDB5E67B2953CB3C7D31150D5479B75.e.482")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.671016C478D89C027E75B16AC2C6E280.d.ECDB5E67B2953CB3C7D31150D5479B75.e.483")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.671016C478D89C027E75B16AC2C6E280.d.ECDB5E67B2953CB3C7D31150D5479B75.e.484")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a5906

In [1625]:
STORE_Menu = []
for i in menu_elements:
    print(i.text)
    STORE_Menu.append(i.text)
split_menu = [item.split(", ") for item in STORE_Menu]
df1 = pd.DataFrame(split_menu, columns=["메뉴"])
split_menu

해산물스페셜5
조개구이(가리비)
육회탕탕이
조개탕
해산물스페셜4
해산물스페셜3
문어숙회
산낙지
전복회
해삼
멍게


[['해산물스페셜5'],
 ['조개구이(가리비)'],
 ['육회탕탕이'],
 ['조개탕'],
 ['해산물스페셜4'],
 ['해산물스페셜3'],
 ['문어숙회'],
 ['산낙지'],
 ['전복회'],
 ['해삼'],
 ['멍게']]

In [1626]:
price_elements = driver.find_elements(By.CSS_SELECTOR, ".GXS1X")
price_elements

[<selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.671016C478D89C027E75B16AC2C6E280.d.ECDB5E67B2953CB3C7D31150D5479B75.e.491")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.671016C478D89C027E75B16AC2C6E280.d.ECDB5E67B2953CB3C7D31150D5479B75.e.492")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.671016C478D89C027E75B16AC2C6E280.d.ECDB5E67B2953CB3C7D31150D5479B75.e.493")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.671016C478D89C027E75B16AC2C6E280.d.ECDB5E67B2953CB3C7D31150D5479B75.e.494")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a59066a029d99552f", element="f.671016C478D89C027E75B16AC2C6E280.d.ECDB5E67B2953CB3C7D31150D5479B75.e.495")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d786338414f4122a5906

In [1627]:
STORE_Price=[]
for i in price_elements:
    print(i.text)
    STORE_Price.append(i.text)
# 쉼표 제거 및 데이터 분리
split_Price = [item.replace(",", "").strip() if item and item.strip() else None for item in STORE_Price]

df2 = pd.DataFrame(split_Price, columns=["가격"])
split_Price

55,000원
55,000원
45,000원
39,000원
45,000원
39,000원
39,000원
35,000원
39,000원
29,000원
29,000원


['55000원',
 '55000원',
 '45000원',
 '39000원',
 '45000원',
 '39000원',
 '39000원',
 '35000원',
 '39000원',
 '29000원',
 '29000원']

In [1628]:
# x축으로 합치기
result = pd.concat([naver_restaurants,df1, df2], axis=1)
result


,식당명,카테고리,주소,영업시간,메뉴,가격
0,바다이야기,조개요리,대구 동구 동부로30길 42-1,영업중24:00에0시0분에`금17:00-24:00`토17:00-24:00`일정기휴무...,해산물스페셜5,55000원
1,NaN,NaN,NaN,NaN,조개구이(가리비),55000원
2,NaN,NaN,NaN,NaN,육회탕탕이,45000원
3,NaN,NaN,NaN,NaN,조개탕,39000원
4,NaN,NaN,NaN,NaN,해산물스페셜4,45000원
5,NaN,NaN,NaN,NaN,해산물스페셜3,39000원
6,NaN,NaN,NaN,NaN,문어숙회,39000원
7,NaN,NaN,NaN,NaN,산낙지,35000원
8,NaN,NaN,NaN,NaN,전복회,39000원
9,NaN,NaN,NaN,NaN,해삼,29000원


In [1629]:
csv4=naver_restaurants['식당명'][0]+"_가게정보.csv"
result.to_csv(csv4,index=False,encoding='utf-8-sig')